In [0]:
!pip install gensim

import gensim
import nltk
from gensim.models import Doc2Vec
from multiprocessing import cpu_count
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt

In [0]:
from gensim.models import Doc2Vec
from multiprocessing import cpu_count

In [3]:
# Uploading files to google collaboratory cloud (Please choose the file on your local system after running this cell)

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
import io
data = io.BytesIO(uploaded['news_articles.csv'])  

import pandas as pd
news_data = pd.read_csv(data)

Saving news_articles.csv to news_articles.csv
User uploaded file "news_articles.csv" with length 8590199 bytes


In [4]:
news_data.head(5)

,Article_Id,Title,Author,Date,Content,URL
0,0,14 dead after bus falls into canal in Telangan...,Devyani Sultania,"August 22, 2016 12:34 IST",At least 14 people died and 17 others were inj...,http://www.ibtimes.co.in/14-dead-after-bus-fal...
1,1,Pratibha Tiwari molested on busy road Saath ...,Suparno Sarkar,"August 22, 2016 19:47 IST",TV actress Pratibha Tiwari who is best known ...,NaN
2,2,US South Korea begin joint military drill ami...,Namrata Tripathi,"August 22, 2016 18:10 IST",The United States and South Korea began a join...,http://www.ibtimes.co.in/us-south-korea-begin-...
3,3,Illegal construction in Bengaluru Will my hou...,S V Krishnamachari,"August 22, 2016 17:39 IST",The relentless drive by Bengaluru s Bangalore...,http://www.ibtimes.co.in/illegal-construction-...
4,4,Punjab Gau Rakshak Dal chief held for assaulti...,Pranshu Rathee,"August 22, 2016 17:34 IST",Punjab Gau Raksha Dal chief Satish Kumar and h...,http://www.ibtimes.co.in/punjab-gau-rakshak-da...


In [0]:
news_list=list()
class LabeledLineSentence(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for uid, line in enumerate(open(filename)):
            yield LabeledSentence(words=line.split(), labels=['SENT_%s' % uid])
            
for index, a in news_data.iterrows():   
    
    sentence =a['Content']+' '+str(a['Title'])+' '+str(a['Author'])
    
    news_list.append(gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(sentence), [a['Article_Id']]))
    

In [0]:
# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 100
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [12]:
model = gensim.models.Doc2Vec( dm = 1, alpha=0.025, size= 50, min_alpha=0.025, min_count=3, workers = num_workers,window = 7,
                              negative = 5, sample = 1e-3,seed = 1)

model.build_vocab(news_list)

# start training
for epoch in range(20):
    if epoch % 2 == 0:
        print ('Now training epoch %s'%epoch)
    model.train(news_list, total_examples= model.corpus_count, epochs=20)
    model.alpha -= 0.002  # decrease the learning rate
    model.min_alpha = model.alpha  # fix the learning rate, no decay

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


Now training epoch 0
Now training epoch 2
Now training epoch 4
Now training epoch 6
Now training epoch 8
Now training epoch 10
Now training epoch 12
Now training epoch 14
Now training epoch 16
Now training epoch 18


In [0]:
%matplotlib inline
import time
import hashlib
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
def optimalK(data, nrefs=99, maxClusters=1000):
    """
    Calculates KMeans optimal K using Gap Statistic from Tibshirani, Walther, Hastie
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[]})
    for gap_index, k in enumerate(range(1, maxClusters)):

        # Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

        # For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.random.random_sample(size=data.shape)
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            refDisp = km.inertia_
            refDisps[i] = refDisp

        # Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_

        # Calculate gap statistic
        gap = np.log(np.mean(refDisps)) - np.log(origDisp)

        # Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
        print({'clusterCount':k, 'gap':gap})
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap}, ignore_index=True)

    return (gaps.argmax() + 1, resultsdf)  # Plus 1 because index of 0 means 1 cluster is optimal, index 2 = 3 clusters are optimal

In [0]:
optimalK(model.docvecs.vectors_docs)   # After around 5-6 clusters, the gap statistic became constant

In [15]:
model.docvecs.vectors_docs

array([[ 3.5091348 , -0.14413981, -4.071165  , ...,  2.213921  ,
         3.2682793 ,  4.252376  ],
       [ 0.63521725, -7.083457  , -0.5158943 , ...,  1.8595085 ,
        -4.546559  ,  3.5689554 ],
       [-4.44898   ,  4.102975  , -1.4453348 , ...,  2.2623215 ,
         0.6280197 , -2.6657956 ],
       ...,
       [-5.555558  ,  0.72939086,  5.965756  , ..., -2.6902337 ,
        -0.5852781 , -2.135957  ],
       [ 1.6006789 , -1.9682505 ,  3.1236882 , ...,  4.9834876 ,
        -0.40947348,  1.5190239 ],
       [-5.9649644 ,  4.1349173 ,  6.1578774 , ...,  4.0252585 ,
         4.8884516 , -4.3150663 ]], dtype=float32)

In [0]:
km = KMeans(20)
a=km.fit(model.docvecs.vectors_docs)

In [17]:
a

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [18]:
a.labels_

array([16, 14,  3, ..., 15,  5, 15], dtype=int32)

In [0]:
clusters = km.labels_.tolist()

In [20]:
clusters[:10]

[16, 14, 3, 16, 14, 10, 9, 17, 16, 16]

In [0]:
news_data['clusters']=clusters

In [22]:
news_data.ix[1419]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


Article_Id                                                 1419
Title         Samsung Galaxy C5  Galaxy C7 price will determ...
Author                                          Raymond Ronamai
Date                                    June 24, 2016 18:33 IST
Content       Samsung has still not revealed if its recently...
URL                                                         NaN
clusters                                                     15
Name: 1419, dtype: object

In [25]:
news_data[news_data.clusters==4]   # Cluster 4 is more on Airlines and Transportation

,Article_Id,Title,Author,Date,Content,URL,clusters
491,491,Comedy Nights with Kapil s last episode not sc...,Ankita Mehta,"January 18, 2016 15:46 IST",The tiff between comedian Kapil Sharma and Col...,the producer,4
1308,1308,TRAFI Public transport assist app debuts in I...,Rohit KVN,"April 14, 2016 13:22 IST",Globally acclaimed public transportation assis...,http://www.ibtimes.co.in/trafi-public-transpor...,4
1438,1438,Pokemon Go update Is the 3-step glitch fixe...,Aditya Bhat,"August 2, 2016 18:05 IST",Pokemon Go the smash hit AR mobile game fro...,NaN,4
1924,1924,Insensitive Air India Staffers Face Social M...,Anu James,"February 17, 2015 14:19 IST",India s national carrier Air India fell victim...,http://www.ibtimes.co.in/insensitive-air-india...,4
2177,2177,India to have 50 more operational airports by ...,Kalyani Pandey,"August 22, 2016 13:20 IST",Taking ahead the government s plan of ensuring...,http://www.ibtimes.co.in/india-have-50-more-op...,4
2178,2178,Govt may spend Rs 6 000 crore to revamp regio...,Kalyani Pandey,"April 25, 2016 14:45 IST",In a bid to boost regional air connectivity t...,NaN,4
2179,2179,Vistara plans using smaller aircraft new bran...,Kalyani Pandey,"July 14, 2016 09:54 IST",India s domestic carrier Vistara is consider...,http://www.ibtimes.co.in/vistara-plans-using-s...,4
2180,2180,India to be 3rd largest civil aviation market ...,Kalyani Pandey,"March 17, 2016 17:19 IST",India is set to become the third largest civil...,http://www.ibtimes.co.in/india-be-3rd-largest-...,4
2181,2181,Govt may scrap 5-year norm for airlines to fly...,Kalyani Pandey,"March 9, 2016 14:54 IST",The government is reportedly planning to do aw...,http://www.ibtimes.co.in/govt-may-scrap-5-year...,4
2187,2187,Ministers meet to discuss new civil aviation p...,Kalyani Pandey,"February 19, 2016 16:33 IST",Union Home Minister Rajnath Singh met Thursday...,http://www.ibtimes.co.in/ministers-meet-discus...,4


In [0]:
user2vec_list = list(news_data.groupby('clusters')['Article_Id'].count())

In [28]:
user2vec_list   #number of elements in each cluster

[250,
 255,
 136,
 236,
 176,
 238,
 249,
 142,
 210,
 229,
 511,
 171,
 310,
 135,
 266,
 309,
 152,
 337,
 260,
 259]

### While giving recommendations to new users, top 5 Most frequently visited articles can be picked up from every cluster and bottom 5 can be most least visited articles in the latent space